In [ ]:
import pandas as pd
import numpy as np
from helpers import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from IPython.display import display
from geopy.distance import geodesic
import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
%load_ext autoreload
%autoreload 2

In [ ]:
ratings = pd.read_csv('dataset/FilteredData/ratings.csv')
users = pd.read_csv('dataset/FilteredData/users.csv')

In [ ]:
#Changed this helper function to get the difference with global mean (ratings and percentage)

def location_style_stats(ratings, users, year):
    year_filter = ratings['year'] == year
    ratings = ratings.loc[year_filter]
    
    ratings = pd.merge(ratings, users[['user_id', 'nbr_ratings', 'location']], on='user_id', how='inner')

    ratings_groupedby_loc_style = ratings.groupby(['location', 'style'])
    ratings_style = ratings.groupby("style")
    total_nb = len(ratings)
    
    location_style = ratings_groupedby_loc_style.size().reset_index(name='number')
    ratings_groupedby_loc = ratings.groupby('location')
    ratings_loc = ratings_groupedby_loc.size().reset_index(name='total_loc_number')

    #Get the counts for each style globally
    
    style_counts = ratings['style'].value_counts()
    global_style = pd.DataFrame({'style': style_counts.index, 'total_number': style_counts.values})
    print(global_style)
    global_style['percentage'] = 100* (global_style['total_number'] / total_nb)
    print(global_style)
    ratings_loc['location_mean'] = ratings_groupedby_loc['rating'].mean().reset_index(name='location_mean')[
        'location_mean']
    ratings_loc['location_std'] = ratings_groupedby_loc['rating'].std().reset_index(name='location_std')[
        'location_std']
    ratings_loc['location_std'] = ratings_loc['location_std'].fillna(1)
    
    ratings_global = pd.DataFrame()
    ratings_global['global_mean'] = ratings_style['rating'].mean()
    ratings_global = ratings_global.reset_index()
   
    
    ratings_loc['normalized_rating'] = ratings.apply(lambda row: abs(row['rating'] - ratings_global.loc[(ratings_global['style'] == row['style'])]['global_mean'].values[0]), axis = 1)
    
    location_style = pd.merge(location_style, ratings_loc, on='location', how='inner')

    location_style['popularity_percentage'] = 100 * (location_style['number'] / location_style['total_loc_number'])

    print(ratings_global)
    location_style['difference_percentage'] = location_style.apply(lambda row: abs(row['popularity_percentage'] - global_style.loc[(global_style['style'] == row['style'])]['percentage'].values[0]), axis = 1)
    
    location_style['mean_rating'] = ratings_groupedby_loc_style['rating'].mean().reset_index(name='mean_rating')['mean_rating']
    location_style = location_style[
        ['location', 'style', 'number', 'total_loc_number', 'popularity_percentage', 'mean_rating', 'location_mean',
         'location_std', 'difference_percentage', 'normalized_rating']]
    location_style['z_score'] = (location_style['mean_rating'] - location_style['location_mean']) / location_style[
        'location_std']
    #location_style = location_style[location_style['number'] >= loc_style_threshold]

    return location_style

In [ ]:
loc = location_style_stats(ratings, users, 2011)

In [ ]:
years = np.arange(2006, 2018)
print(years)
style_popularities_by_year = pd.DataFrame()
for year in years:
    print(year)
    location_style = location_style_stats(ratings.copy(), users.copy(), year)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_most_popular = location_style_groupedby_loc['difference_percentage'].idxmax()
    most_popular_styles_by_location = location_style.loc[idx_most_popular]
    most_popular_styles_by_location = most_popular_styles_by_location[['location', 'style']]
    most_popular_styles_by_location.set_index('location', inplace = True)
    most_popular_styles_by_location = most_popular_styles_by_location.rename(columns = {'style': str(year)})
    style_popularities_by_year = pd.concat([style_popularities_by_year, most_popular_styles_by_location], axis = 1)

